In [1]:
import h5py

def inspect_h5(path):
    """
    Mở file .h5 tại đường dẫn `path`,
    liệt kê các dataset, shape và dtype của chúng.
    """
    with h5py.File(path, 'r') as f:
        keys = list(f.keys())
        print(f"Datasets in {path!r}: {keys}\n")
        for k in keys:
            ds = f[k]
            print(f"  • {k:12s} shape={ds.shape}, dtype={ds.dtype}")

if __name__ == "__main__":
    file_path = "data/ACDC2017/ACDC_for2D/TestSet_volumes/patient004_frame15.h5"  # ví dụ
    inspect_h5(file_path)


Datasets in 'data/ACDC2017/ACDC_for2D/TestSet_volumes/patient004_frame15.h5': ['image', 'label']

  • image        shape=(10, 256, 232), dtype=float64
  • label        shape=(10, 256, 232), dtype=uint8


In [2]:
import SimpleITK as sitk

# Thay đường dẫn bằng file bạn vừa download
nii_path = "patient004_frame15.nii.gz"

img = sitk.ReadImage(nii_path)
spacing   = img.GetSpacing()    # (spacing_x, spacing_y, spacing_z)
origin    = img.GetOrigin()     # (origin_x, origin_y, origin_z)
direction = img.GetDirection()  # 9 số thể hiện ma trận xoay

size = img.GetSize()            # (size_x, size_y, size_z)
array = sitk.GetArrayFromImage(img)  
print("Array shape (Z, Y, X):", array.shape)

print("Size (X, Y, Z):", size)
print("Spacing (X, Y, Z):", spacing)
print("Origin  (X, Y, Z):", origin)
print("Direction:", direction)


Array shape (Z, Y, X): (10, 256, 232)
Size (X, Y, Z): (232, 256, 10)
Spacing (X, Y, Z): (1.3671875, 1.3671875, 10.0)
Origin  (X, Y, Z): (0.0, 0.0, 0.0)
Direction: (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)


NiftiImageIO (0x25d4f50): patient004_frame15.nii.gz has unexpected scales in sform

NiftiImageIO (0x25d4f50): patient004_frame15.nii.gz has unexpected scales in sform



In [3]:
import SimpleITK as sitk

# Thay đường dẫn bằng file bạn vừa download
nii_path = "patient093_frame01.nii.gz"

img = sitk.ReadImage(nii_path)
spacing   = img.GetSpacing()    # (spacing_x, spacing_y, spacing_z)
origin    = img.GetOrigin()     # (origin_x, origin_y, origin_z)
direction = img.GetDirection()  # 9 số thể hiện ma trận xoay

size = img.GetSize()            # (size_x, size_y, size_z)
array = sitk.GetArrayFromImage(img)  
print("Array shape (Z, Y, X):", array.shape)

print("Size (X, Y, Z):", size)
print("Spacing (X, Y, Z):", spacing)
print("Origin  (X, Y, Z):", origin)
print("Direction:", direction)


Array shape (Z, Y, X): (10, 224, 180)
Size (X, Y, Z): (180, 224, 10)
Spacing (X, Y, Z): (1.5625, 1.5625, 7.0)
Origin  (X, Y, Z): (0.0, 0.0, 0.0)
Direction: (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)


NiftiImageIO (0x26f8790): patient093_frame01.nii.gz has unexpected scales in sform

NiftiImageIO (0x26f8790): patient093_frame01.nii.gz has unexpected scales in sform



In [6]:
import os

def find_org_nii(case_name, root_for2d):
    """
    Tìm file {case_name}.nii.gz trong thư mục images_N, phù hợp với cấu trúc:
      root_for2d.replace('_for2D','')/images_N/{case_name}.nii.gz
    hoặc
      …/images_N/patientXXX/{case_name}.nii.gz

    Args:
        case_name (str): ví dụ "patient004_frame15"
        root_for2d (str): ví dụ ".../ACDC_for2D"

    Returns:
        str: full path đến .nii.gz
    """
    # 1) Lấy thư mục chứa ảnh gốc (không for2D)
    base = root_for2d.replace("_for2D", "")
    images_n = os.path.join(base, "images_N")

    # 2) thử tìm trực tiếp
    direct = os.path.join(images_n, f"{case_name}.nii.gz")
    if os.path.exists(direct):
        return direct

    # 3) thử tìm trong từng subfolder (patientXXX/)
    for sub in os.listdir(images_n):
        subdir = os.path.join(images_n, sub)
        if not os.path.isdir(subdir):
            continue
        candidate = os.path.join(subdir, f"{case_name}.nii.gz")
        if os.path.exists(candidate):
            return candidate

    # 4) không tìm thấy
    raise FileNotFoundError(
        f"Không tìm thấy {case_name}.nii.gz trong {images_n} hoặc các subfolder của nó."
    )


In [14]:
case_name = "patient004_frame15"
root_for2d = "data/ACDC2017/ACDC_for2D"
org_nii = find_org_nii(case_name, root_for2d)
print("Found org nii at:", org_nii)


Found org nii at: data/ACDC2017/ACDC/images_N/patient004/patient004_frame15.nii.gz


In [16]:
import SimpleITK as sitk

org_itk = sitk.ReadImage(org_nii)
spacing = org_itk.GetSpacing()
print("Spacing:", spacing)

Spacing: (1.3671875, 1.3671875, 10.0)


NiftiImageIO (0x343c770): data/ACDC2017/ACDC/images_N/patient004/patient004_frame15.nii.gz has unexpected scales in sform

NiftiImageIO (0x343c770): data/ACDC2017/ACDC/images_N/patient004/patient004_frame15.nii.gz has unexpected scales in sform

